In [1]:
import os
import codecs
import re
import time
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
from gensim.test.utils import common_texts

from gensim.models import Word2Vec

In [2]:
stopwords = ["баш","без","биће","био","бити","близу","број","дана","данас","доћи","добар","добити","док","доле","дошао","други","дуж","два","често","чији","где","горе","хвала","и","ићи","иако","иде","има","имам","имао","испод","између","изнад","изван","изволи","један","једини","једном","јесте","још","јуче","кад","како","као","кога","која","које","који","кроз","мали","мањи","мисли","много","моћи","могу","мора","морао","на","наћи","наш","негде","него","некад","неки","немам","нешто","није","ниједан","никада","нисмо","ништа","њега","његов","њен","њих","њихов","око","около","она","онај","они","оно","осим","остали","отишао","овако","овамо","овде","ове","ово","о","питати","почетак","поједини","после","поводом","правити","пре","преко","према","први","пут","радије","сада","смети","шта","ствар","стварно","сутра","сваки","све","свим","свугде","тачно","тада","тај","такође","тамо","тим", "у", "учинио","учинити","умало","унутра","употребити","узети","ваш","већина","веома","видео","више","захвалити","зашто","због","желео","жели","знати"]

In [11]:
#not using this one
dataset_path = '../acts'
dataset = {}
TAG_RE = re.compile(r'<[^>]+>')
for file_path in os.listdir(dataset_path):
    with codecs.open(os.path.join(dataset_path, file_path), 'r', encoding='utf-8') as f:
        data = f.read()
        data = data.lower() #lowercase
        data = TAG_RE.sub('', data) #remove html tags
        data = data.translate(str.maketrans('', '', string.punctuation + "„”")) #remove punctuation
        data_split = data.split() #remove extra whitespaces
        data_split = [word for word in data_split if word not in stopwords and not word.isnumeric()]
#         data = " ".join(data_split)
        dataset[file_path] = data_split

In [3]:
#new
def hasNumbers(inputString):
     return any(char.isdigit() for char in inputString)

In [4]:
#new
clan_re = re.compile("<p>[ \t\n]*Члан[ \t\n]*[0-9]*\..*?<\/p>")
clan_re.split("sadsadasd<p>Члан  13.  </p>asdj<p>Члан  12.  </p>sadsad")

['sadsadasd', 'asdj', 'sadsad']

In [20]:
#using this one to load the dataset instead
start = time.time()
dataset_path = '../acts'
dataset = {}
TAG_RE = re.compile(r'<[^>]+>')
corpus_clans = []
for file_path in os.listdir(dataset_path):
    with codecs.open(os.path.join(dataset_path, file_path), 'r', encoding='utf-8') as f:
        data = f.read()
        clans = clan_re.split(data)
        clans_no_html = [TAG_RE.sub('', clan) for clan in clans] #remove html tags
        clans_no_white_spaces = [' '.join(clan.split()) for clan in clans_no_html]
        corpus_clans.append(clans_no_white_spaces)
        clans_no_punc = (clan.translate(str.maketrans('', '', string.punctuation + "„”–vi")) for clan in clans_no_white_spaces) #remove punctuation
        clans_lower = [clan.lower() for clan in clans_no_punc] #lowercase
        clans_no_stopwords = []
        for i, clan in enumerate(clans_lower):
            new_clan = clan.split()
            new_clan = [word for word in new_clan if word not in stopwords and not hasNumbers(word)]
            new_clan = ' '.join(new_clan)
            clans_no_stopwords.append(new_clan)
#         data = [row_clean for row in data if (row_clean := " ".join(row.split())) != ''] #remove extra whitespaces
        clans_clean = [" ".join(clan.split()) for clan in clans_no_stopwords] #remove extra whitespaces
        clans_cut = [clan[:500000] for clan in clans_clean]
        dataset[file_path] = clans_cut
end = time.time()
corpus = list(dataset.values())
print(end-start)

27.953356504440308


In [6]:
#new
sentences = [clan.split(" ") for zakon in corpus for clan in zakon]

In [7]:
# corpus = list(dataset.values()) #this line is already done before
#changed next line
model = Word2Vec(sentences=sentences, size=300, window=5, min_count=1, workers=4)

In [10]:
#moved up, it was 2 cells before
def query_to_vec(query, model):
    query = query.lower() #lowercase
    query = TAG_RE.sub('', query) #remove html tags
    query = query.translate(str.maketrans('', '', string.punctuation + "„”")) #remove punctuation
    query_split = query.split() #remove extra whitespaces
    query_split = [word for word in query_split if word not in stopwords and not word.isnumeric()]
    wordvecs = []
    for word in query_split:
        try:
            wordvecs.append(model.wv[word])
        except KeyError:
#             print(f'{word} not in vocabulary')
            continue
    if wordvecs:
        wordvecs_mean = np.array(wordvecs).sum(axis=0) / float(len(wordvecs))
        return wordvecs_mean
    else:
        raise ValueError('No words from the query are present in the model vocabulary')        

In [11]:
#changed
def dataset_to_vectors(dataset, model):
    """Returns dataset vectorized. Doesn't change the dataset"""
    X = []
    for key, clans in dataset.items():
        try:
            clans_embedding = [query_to_vec(clan, model) for clan in clans]
        except ValueError:
            clans_embedding = [[]]
#         for clan in clans:    
#             clan_embedding = query_to_vec(clan, model)
#             clans_embedding.append(clan_embedding)
        X.append(clans_embedding)
    print(len(X))
    return X

In [13]:
embeddings = dataset_to_vectors(dataset, model)

1130


In [9]:
#not using this one
def calculate_similarity_word2vec(X, model, query, top_k=10):
    """ Vectorizes the `query` via `query_to_vec` and calculates the cosine similarity of
    the `query` and `X` (all the documents) and returns the `top_k` similar documents."""

    # Vectorize the query to the same length as documents
    query_vec = query_to_vec(query, model)
    query_vec = query_vec.reshape(1,-1)
    # Compute the cosine similarity between query_vec and all the documents
    cosine_similarities = cosine_similarity(X,query_vec).flatten()
    # Sort the similar documents from the most similar to less similar and return the indices
    most_similar_doc_indices = np.argsort(cosine_similarities, axis=0)[:-top_k-1:-1]
    return (most_similar_doc_indices, cosine_similarities)

In [36]:
#new
def calculate_similarity(embeddings, model, query, top_documents=10, top_sentences=3):
    """ Embeds the `query` via `model` and calculates the cosine similarity of
    the `query` and `X` (all the documents) and returns the `top_k` similar documents."""

    # Vectorize the query to the same length as sentences in documents
    query_vec = query_to_vec(query, model)
    query_vec = query_vec.reshape(1,-1)
    # Compute the cosine similarity between query_vec and all the documents
    scores = np.array([])
    top_sentence_indices = []
    for i, zakon_embedding in enumerate(embeddings):
        if len(zakon_embedding[0]) > 0:
            cosine_scores = cosine_similarity(query_vec, zakon_embedding)[0]
        else:
            cosine_scores = np.array([0.])
        indices_best = cosine_scores.argsort()[-1:-top_sentences-1:-1]
        scores = np.append(scores, cosine_scores[indices_best].mean())
        top_sentence_indices.append(indices_best)
    # Sort the similar documents from the most similar to less similar and return the indices
    most_similar_doc_indices = np.argsort(scores, axis=0)[-1:-top_documents-1:-1].astype(int)
    top_sentence_indices = np.array(top_sentence_indices)[most_similar_doc_indices]
    return (most_similar_doc_indices, scores, top_sentence_indices)

In [15]:
def show_similar_documents(df, cosine_similarities, similar_doc_indices):
    """ Prints the most similar documents using indices in the `similar_doc_indices` vector."""
    counter = 1
    for index in similar_doc_indices:
        print('Top-{}, Similarity = {}'.format(counter, cosine_similarities[index]))
        print('body: {}, '.format(df[index]))
        print()
        counter += 1

In [18]:
#new 
def show_similar_documents(doc_titles, cosine_similarities, similar_doc_indices, corpus, similar_doc_sentence_indices):
    """ Prints the most similar documents using indices in the `similar_doc_indices` vector."""
    counter = 1
    
    for i, index in enumerate(similar_doc_indices):
        print('Top-{}, Similarity = {}'.format(counter, cosine_similarities[index]))
        print('body: {}, '.format(doc_titles[index]))
        print('Most similar articles: ')
        for jandex in similar_doc_sentence_indices[i]:
            print(f'\t{corpus[index][jandex]}')
        print()
        counter += 1

In [38]:
#changed
top_documents = 10
top_sentences = 3
user_question = 'безбедност на раду'
search_start = time.time()
sim_vecs, cosine_similarities, top_sentence_indices = calculate_similarity(embeddings, 
                                                                           model, 
                                                                           user_question, 
                                                                           top_documents=top_documents,
                                                                           top_sentences=top_sentences)
search_time = time.time() - search_start
print("search time: {:.2f} ms".format(search_time * 1000))
print()
show_similar_documents(list(dataset.keys()), cosine_similarities, sim_vecs, corpus_clans, top_sentence_indices)

search time: 438.04 ms

Top-1, Similarity = 0.6951673626899719
body: 432.html, 
Most similar articles: 
	Послодавац је дужан да представнику запослених, односно Одбору омогући: 1) увид у све акте који се односе на безбедност и здравље на раду; 2) да учествују у разматрању свих питања која се односе на безбедност и здравље на раду, да предлажу и буду консултовани. Послодавац је дужан да представника запослених, односно Одбор информише о свим подацима који се односе на безбедност и здравље на раду. *Службени гласник РС, број 91/2015
	У саставу министарства надлежног за рад образује се Управа за безбедност и здравље на раду, која обавља послове државне управе са циљем унапређивања и развоја безбедности и здравља на раду, односно смањења повреда на раду, професионалних обољења и обољења у вези са радом (у даљем тексту: Управа).
	Овим законом уређује се спровођење и унапређивање безбедности и здравља на раду лица која учествују у радним процесима, као и лица која се затекну у радној околини

<ipython-input-36-383e26706464>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  top_sentence_indices = np.array(top_sentence_indices)[most_similar_doc_indices]


In [32]:
query = '''
о давању контрагаранције Републике Србије Србији и Црној Гори по кредиту Међународног удружења за развој (Кредит за развој приватног и финансијског сектора)
'''